# 네이버 지식인 크롤링

In [35]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [40]:
chromedriver = '/Users/sku11/Downloads/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)


In [41]:
base_url = 'https://kin.naver.com/search/list.naver?query='
url = base_url + quote('여친선물')
driver.get(url)


In [42]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


In [43]:
lis = soup.select('ul.basic1 > li')
len(lis)

10

In [44]:
li = lis[0]
sub_url = li.select_one('a._searchListTitleAnchor')['href']
sub_url

'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80101&docId=395035694&qb=7Jes7Lmc7ISg66y8&enc=utf8&section=kin&rank=1&search_sort=0&spq=0'

In [45]:
driver.get(sub_url)


In [46]:
#
count = int(driver.find_element_by_css_selector('._answerCount.num').text)
count


8

In [47]:
answer_page = (count -1) // 5
print(count, answer_page)


8 1


In [48]:
for i in range(answer_page):
    driver.find_element_by_id('nextPageButton').click()
    time.sleep(2)

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [51]:
divs = soup.select('div.answer-content__item._contentWrap._answer')
len(divs)

0

In [53]:
div = divs[0]
ps = div.select('p.se-text-paragraph')
for p in ps:
    print(p.get_text())

IndexError: list index out of range

In [ ]:
driver.back()

#위에는 한페이지 크롤링
#아래부터는 for 루프    

In [33]:
answer_list = []
for li in lis:
    sub_url = li.select_one('a._searchListTitleAnchor')['href']
    driver.get(sub_url)
    time.sleep(2)

    count = int(driver.find_element_by_css_selector('._answerCount.num').text)
    answer_page = (count - 1) // 5
    print(count, end=', ')
    for i in range(answer_page):
        driver.find_element_by_id('nextPageButton').click()
        time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    divs = soup.select('div.answer-content__item._contentWrap._answer')
    print(len(divs))
    for div in divs:
        ps = div.select('p.se-text-paragraph')
        for p in ps:
            text = p.get_text()
            if text == ' ' or text == '\u200b':
                continue
            answer_list.append(text)

    driver.back()
    time.sleep(2)

In [ ]:
answer_list[100:110]

## 1~10 페이지 데이터 한꺼번에 하기 

In [ ]:
answer_list = []
for page in range(1,11):
    print('page', page)
    url = f"{base_url}{quote('여친선물')}&page={page}"
    driver.get(url)
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    lis = soup.select('ul.basic1 > li')

    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        driver.get(sub_url)
        time.sleep(2)

        count = int(driver.find_element_by_css_selector('._answerCount.num').text)
        answer_page = (count - 1) // 5
        #print(count, end=', ')
        for i in range(answer_page):
            driver.find_element_by_id('nextPageButton').click()
            time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        divs = soup.select('div.answer-content__item._contentWrap._answer')
        #print(len(divs))
        for div in divs:
            ps = div.select('p.se-text-paragraph')
            for p in ps:
                text = p.get_text()
                if text == ' ' or text == '\u200b':
                    continue
                answer_list.append(text)

        driver.back()
        time.sleep(2)

In [ ]:
len(answer_list)


In [ ]:
with open('new_gift.txt', 'w', encoding='utf-8') as fp:
    for answer in answer_list:
        fp.write(answer+'\n')

In [ ]:
driver.close()